In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Oct_weekend")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [2]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e3
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [3]:
# getting the coefficients of the costs
coeffs_dict_Oct_weekend = readall("../temp_files/coeffs_dict_Oct_weekend.json")
coeffs_dict_Oct_weekend = JSON.parse(coeffs_dict_Oct_weekend)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [4:8]
c_grid = .5:.5:3.
lamb_grid = [10. .^(-3:4)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Oct_weekend["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Oct_weekend["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Oct_weekend["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [4]:
lamb_grid

8-element Array{Float64,1}:
     0.001
     0.01 
     0.1  
     1.0  
    10.0  
   100.0  
  1000.0  
 10000.0  

In [5]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Oct_weekend.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [6]:
xl

Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}} with 720 entries:
  (8,2.5,10.0,3)    => [1180.2177921959992,755.3575276939998,1050.8177318040007…
  (4,1.0,0.01,2)    => [1162.369508003999,795.7062735159996,1068.6660159960009,…
  (4,3.0,0.001,1)   => [1706.742175859999,582.958341,524.2933481399994,1834.033…
  (8,2.0,1000.0,3)  => [1180.2177921959992,757.1915615949999,1050.8177318040007…
  (6,1.5,0.1,1)     => [2231.035524,582.958341,0.0,1834.033901,2231.035524,643.…
  (6,1.5,100.0,1)   => [1182.4488277199991,744.353324287999,1048.5866962800008,…
  (4,2.0,1000.0,3)  => [1222.6074671519996,637.9793580299989,1008.4280568479988…
  (5,2.5,10.0,1)    => [1218.1453961039986,650.8175953370014,1012.8901278959993…
  (7,0.5,1.0,3)     => [1142.2901882880005,861.148535623,1088.7453357119994,155…
  (5,0.5,100.0,2)   => [1157.9074369560008,812.2125786249997,1073.1280870439991…
  (7,1.5,1.0,3)     => [1173.5246856240005,773.6978667039995,1057.5108383759994…
  (4,0.5,0.01,1)    => [1189.141934292

In [7]:
xl[(8,2.5,0.1,2)]

24-element Array{Float64,1}:
 1057.51 
 1021.96 
 1173.52 
 1395.03 
 1057.51 
 1084.14 
 1179.37 
 1276.83 
 1558.7  
 1154.79 
 1073.94 
  697.608
 1087.75 
  906.439
 1056.65 
  732.115
 1219.94 
 1272.42 
 2094.78 
 1613.39 
  936.856
  710.033
 1188.24 
 1263.71 